# Merge all auctions into one

Combines several datasets and do some simple analyses.  


### User variables

In [1]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
AuctionDates = ['2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12',
                '2018-01','2018-02','2018-03','2018-04','2018-05','2018-06','2018-07','2018-08','2018-09','2018-10','2018-11','2018-12',
                '2019-01','2019-02','2019-03','2019-04']

### Import modules 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
plt.style.use('ggplot')

### Load all data

Auctions are saved monthly in a .pkl file

In [3]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    fn = '../data/rdw-data-{:s}.pkl'.format(
        AuctionDate
    )
    print(fn)
    df = pd.read_pickle(fn)
    data[AuctionDate] = df

print('\nlast rows of last file')
data[AuctionDates[-1]].tail()

load data
../data/rdw-data-2017-03.pkl
../data/rdw-data-2017-04.pkl
../data/rdw-data-2017-05.pkl
../data/rdw-data-2017-06.pkl
../data/rdw-data-2017-07.pkl
../data/rdw-data-2017-08.pkl
../data/rdw-data-2017-09.pkl
../data/rdw-data-2017-10.pkl
../data/rdw-data-2017-11.pkl
../data/rdw-data-2017-12.pkl
../data/rdw-data-2018-01.pkl
../data/rdw-data-2018-02.pkl
../data/rdw-data-2018-03.pkl
../data/rdw-data-2018-04.pkl
../data/rdw-data-2018-05.pkl
../data/rdw-data-2018-06.pkl
../data/rdw-data-2018-07.pkl
../data/rdw-data-2018-08.pkl
../data/rdw-data-2018-09.pkl
../data/rdw-data-2018-10.pkl
../data/rdw-data-2018-11.pkl
../data/rdw-data-2018-12.pkl
../data/rdw-data-2019-01.pkl
../data/rdw-data-2019-02.pkl
../data/rdw-data-2019-03.pkl
../data/rdw-data-2019-04.pkl

last rows of last file


,Source,Title,Price,Draw,Raw_text,N_images,Images,Note,LotNr,LotType,...,no_regneeded,rhd,no_odo,no_road,disclaim_cr6,crewcab,carwrap,no_vin,SupInfo,rdwinfo
2019-4-7347,http://www.domeinenrz.nl/catalogus/verkoop_bij...,Kavel K1900047347,488.0,False,"[Type h434, Niet kentekenplichtig, Zonder kent...",3,[http://www.domeinenrz.nl/ufc/static/155360666...,False,K1900047347,Aanhangwagen (Enkelasser),...,True,False,False,False,False,False,False,False,NaN,{'TimeStamp': '20190407'}
2019-4-7348,http://www.domeinenrz.nl/catalogus/verkoop_bij...,Kavel K1900047348,24590.0,False,"[Type ft xf 105, Kenteken BV-XS-15, Afgelezen ...",3,[http://www.domeinenrz.nl/ufc/static/155360666...,False,K1900047348,Vrachtwagencombinatie (Trekker),...,False,False,True,False,False,False,False,False,\nOplegger\nVAN HOOL\nType t-315\nKenteken OB-...,"{'aantal_cilinders': {0: 6}, 'aantal_wielen': ..."
2019-4-7349,http://www.domeinenrz.nl/catalogus/verkoop_bij...,Kavel K1900047349,3115.0,False,"[Type fm12 4x2t fal7.1 rad-a4; high, Automatic...",3,[http://www.domeinenrz.nl/ufc/static/155360666...,False,K1900047349,Trekker,...,False,False,False,False,False,False,False,False,NaN,{'TimeStamp': '20190407'}
2019-4-7350,http://www.domeinenrz.nl/catalogus/verkoop_bij...,Kavel K1900047350,3689.0,False,"[Inhoud 22000 liter, De kavelomschrijving op d...",2,[http://www.domeinenrz.nl/ufc/static/155360666...,False,K1900047350,Tankcontainer,...,False,False,False,False,False,False,False,False,\nInhoud 22000 liter,{'TimeStamp': '20190407'}
2019-4-7351,http://www.domeinenrz.nl/catalogus/verkoop_bij...,Kavel K1900047351,3971.0,False,"[Type ae 45 ce, Nederlands kenteken, Afgelezen...",3,[http://www.domeinenrz.nl/ufc/static/155360666...,False,K1900047351,Bedrijfswagen,...,False,False,False,False,False,False,False,False,\nDit voertuig staat voor export geregistreerd...,{'TimeStamp': '20190407'}


### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [4]:
# Images

# print first lot
print('Fields with images look like this:\n')
print('{:7s} | {:7s} | {:14s} | {:7s}\n{:7s}-+-{:s}-+-{:s}-+-{:s}'.format('Month','Column','Type','1st row','-'*7,'-'*7,'-'*14,'-'*46))
for k in data:
    col_idx = pd.np.where([c.lower() in ['image','images'] for c in data[k].columns])[0]
    example = data[k].iloc[0,col_idx].values[0]
    print('{:7s} | {:7s} | {}'.format(k,data[k].columns[col_idx][0],type(example)),end='')
    if type(example) == str:
        print('  | "{} .. {}"'.format(example[0:20],example[-20:]))
    else:
        print(' | length {} '.format(len(example)))

Fields with images look like this:

Month   | Column  | Type           | 1st row
--------+---------+----------------+-----------------------------------------------
2017-03 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-04 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-05 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-06 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-07 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-08 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-09 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-10 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-11 | Images  | <class 'str'>  | "[http://www.domeinen .. /1024/768/image.jpg]"
2017-12 | Images  | <class 'str'>  | "[http://www.domeinen .. /1024/768/image.j

In [5]:
# convert string representation of list to real list
# "['image1', 'image2']"
for k in ['2017-03',
          '2017-04',
          '2017-05',
          '2017-06',
          '2017-07',
          '2017-08',
          '2017-09',
          '2017-10']:
    data[k]['Images'] = data[k]['Image'].apply(eval)
    data[k] = data[k].drop(columns=['Image'])

# convert string representation of list without quotes to real list
# "[image1, image2]"
for k in ['2017-11',
          '2017-12',
          '2018-01',
          '2018-02',
          '2018-03',
          '2018-04']:
    data[k]['Images'] = data[k]['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
    

In [6]:
# Other column name that changed in november 2017
for k in ['2017-03',
          '2017-04',
          '2017-05',
          '2017-06',
          '2017-07',
          '2017-08',
          '2017-09',
          '2017-10']:
    data[k] = data[k].rename(columns={
        'draw':'Draw',
        'nr':'LotNr',
        'raw':'Raw_text',
        'misc':'SupInfo',
        'footnote':'Note',
        'jfq':'jfc',
    })

# Merge all auctions into one data frame

In [7]:
# append data into one data frame
for i,k in enumerate(sorted(data.keys(),reverse=False)):
    if i == 0:
        df = data[k]
    else:
        oldcols = pd.np.setdiff1d(df.columns,data[k].columns)
        newcols = pd.np.setdiff1d(data[k].columns,df.columns)
        if oldcols.size > 0:
            print('fields not present in {:s}'.format(k))
            [print('\t{:s}'.format(c)) for c in oldcols]
        if newcols.size > 0:
            print('fields new in {:s}'.format(k))
            [print('\t{:s}'.format(c)) for c in newcols]
        
        df=df.append(data[k],sort=False).copy()
    print(k,df.shape)
            

2017-03 (234, 52)
2017-04 (434, 52)
2017-05 (561, 52)
2017-06 (804, 52)
2017-07 (993, 52)
2017-08 (1238, 52)
2017-09 (1447, 52)
2017-10 (1626, 52)
fields not present in 2017-11
	disclaim3
	disclaim4
	hybrid
	import
	jfc
	no_igk
	no_nlreg193
	taxi
fields new in 2017-11
	N_images
	Source
	early_reg
	locked
	used_parts
	wo_frame
2017-11 (1877, 58)
fields not present in 2017-12
	disclaim3
	disclaim4
	hybrid
	import
	jfc
	lpg
	no_nlreg193
	no_orireg
	taxi
	used_parts
2017-12 (2118, 58)
fields not present in 2018-01
	disclaim3
	disclaim4
	jfc
	locked
	no_nlreg193
	no_orireg
	taxi
	used_parts
fields new in 2018-01
	disclaim_cr6
	import22_btw21
	import27_btw21
2018-01 (2393, 61)
fields not present in 2018-02
	disclaim3
	disclaim4
	disclaim_cr6
	electric
	import22_btw21
	import27_btw21
	jfc
	locked
	lpg
	no_nlreg193
	no_orireg
	no_regneeded
	taxi
	used_parts
2018-02 (2610, 61)
fields not present in 2018-03
	disclaim3
	disclaim4
	import
	import22_btw21
	import27_btw21
	jfc
	locked
	no_igk
	no_nl

# Continue with cars only

In [8]:
# select data
car=df[df.LotType == 'Personenauto'].copy()

In [9]:
# print lots that are cars per auction
prev = ['1999','0','0']
print('Car lots\nYear | Month | First ... Last index')
for ix in car.index:
    y,m,k = ix.split('-')
    if (y != prev[0]) or (m != prev[1]):
        if prev[0] == '1999':
            print('-----+-------+---------------------',end='')
        else:
            print('.. {:s}'.format(prev[2]),end='')
        print('\n{:4s} | {:>5s} |'.format(y,m),end='')
        prev[0] = y
        prev[1] = m
        print(' {:>5s} .'.format(k),end='')
    prev[2] = k
print('.. {:s}'.format(prev[2]),end='')
car.tail()

Car lots
Year | Month | First ... Last index
-----+-------+---------------------
2017 |     3 |  2000 ... 8318
2017 |     4 |  2000 ... 7309
2017 |     5 |  2200 ... 8025
2017 |     6 |  2000 ... 7311
2017 |     7 |  2200 ... 8178
2017 |     8 |  2001 ... 7304
2017 |     9 |  2200 ... 8187
2017 |    10 |  2000 ... 7171
2017 |    11 |  2200 ... 8305
2017 |    12 |  2000 ... 7308
2018 |     1 |  2200 ... 8170
2018 |     2 |  2000 ... 7184
2018 |     3 |  2200 ... 8338
2018 |     4 |  2000 ... 7336
2018 |     5 |  2200 ... 8226
2018 |     6 |  2000 ... 7315
2018 |     7 |  2200 ... 8311
2018 |     8 |  2000 ... 7328
2018 |     9 |  2200 ... 8183
2018 |    10 |  2000 ... 7319
2018 |    11 |  2200 ... 8326
2018 |    12 |  2000 ... 7361
2019 |     1 |  2201 ... 8329
2019 |     2 |  2001 ... 7264
2019 |     3 |  2200 ... 8318
2019 |     4 |  2000 ... 7314

,LotNr,Price,Title,LotCat,LotType,ItemBrand,ItemType,Mfdate,Mfyear,APKdate,...,wo_frame,used_parts,disclaim_cr6,import22_btw21,import27_btw21,OdoKM_num,disclaim12,crewcab,carwrap,no_vin
2019-4-7212,K1900047212,1738.0,Kavel K1900047212,Voertuigen en onderdelen,Personenauto,FORD,focus,17.06.2008,,,...,False,NaN,False,NaN,NaN,NaN,NaN,False,False,False
2019-4-7213,K1900047213,1919.0,Kavel K1900047213,Voertuigen en onderdelen,Personenauto,SKODA,fabia,02.11.2010,,,...,False,NaN,False,NaN,NaN,NaN,NaN,False,False,False
2019-4-7216,K1900047216,2052.0,Kavel K1900047216,Voertuigen en onderdelen,Personenauto,KIA,rio,03.12.2011,,09.11.2019,...,False,NaN,False,NaN,NaN,NaN,NaN,False,False,False
2019-4-7217,K1900047217,921.0,Kavel K1900047217,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,fox; 40 kw,11.08.2006,,27.08.2019,...,False,NaN,False,NaN,NaN,NaN,NaN,False,False,False
2019-4-7314,K1900047314,2400.0,Kavel K1900047314,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,caddy,10.02.2009,,,...,False,NaN,False,NaN,NaN,NaN,NaN,False,False,False


# Save file

In [10]:
# save data
fn = '../data/cars-from-all-auctions.pkl'
print(fn)
car.to_pickle(fn)

../data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [11]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [12]:
fn = '../assets/example-table-of-cars.md'
example = car.tail(10).copy()
# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'
table_text = pandas_df_to_markdown_table(example)
with open(fn,'w') as file:
    file.write(table_text)
    
print('A markdown table is available as\n\t{}'.format(fn))

A markdown table is available as
	../assets/example-table-of-cars.md
